Experiment with OpenCV for finding the pool table edges within an image.

This version assumes a full-sized vertical image of the table. It is looking
for the marker pattern along the side of the table.

In [ ]:
import cv2 as cv
import numpy as np
import numpy.linalg as linalg
import matplotlib.pyplot as plt
import math
import copy
from Utils import *

In [ ]:
# Handy circle class
class Circle:
    def __init__(self, x,y,r):
        self.x = int(x)
        self.y = int(y)
        self.r = int(r)
        
    def center(self):
        return (self.x, self.y)
    
    def radius(self):
        return self.r

# Handy line class
class LineP:
    def __init__(self, circleA, circleB):
        x1 = circleA.center()[0]
        y1 = circleA.center()[1]
        x2 = circleB.center()[0]
        y2 = circleB.center()[1]
        #print("{0} - {1}".format(y2,y1))
        rise = float(y2 - y1)
        run  = float(x2 - x1)
        if run != 0.0:
            self.m = rise/run
        else:
            self.m = float(10E10)
        self.b = float(y1 - self.m * x1)
        self.radius = float(circleA.radius() + circleB.radius()) / 2.0
        self.circles = []
        self.circles.append(circleA)
        self.circles.append(circleB)
    
    def addCircle(self, circle):
        # scan for duplicates
        for c in self.circles:
            if c.radius() == circle.radius() and c.center()[0] == circle.center()[0] and c.center()[1] == circle.center()[1]:
                return
        self.circles.append(circle)
        
    def val(self):
        return self.m, self.b
        
    def angleRad(self):
        return np.arctan(self.m)
    
    def angleDeg(self):
        return self.angleRad() * 180.0 / math.pi
    
    def pointCount(self):
        return len(self.circles)
    
    def circleRadius(self):
        return self.radius
        
    def distance(self, point):
        a = self.m
        b = -1.0
        c = self.b        
        x = point[0]
        y = point[1]
        d = abs(a*x + b*y + c) / math.sqrt(a*a + b*b)
        return float(d)
    
    def endpoints(self):
        # Scan for the farthest points
        d2Max = 0
        for i in range(0, len(self.circles)-1):
            for j in range(i+1, len(self.circles)):
                dx = self.circles[i].center()[0] - self.circles[j].center()[0]
                dy = self.circles[i].center()[1] - self.circles[j].center()[1]
                d2 = dx*dx + dy*dy
                if d2 > d2Max:
                    d2Max = d2
                    p1 = self.circles[i].center()
                    p2 = self.circles[j].center()
        return p1,p2
    
    def length(self):
        # distance between endpoints
        p1,p2 = self.endpoints()
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        return math.sqrt(dx*dx + dy*dy)
    
    
# Test the endpoints
c1 = Circle(0.0,0.0,1.0)
c2 = Circle(10.0,0.0,1.0)
c3 = Circle(20.0,0.0,1.0)
line = LineP(c1, c2)
pt1, pt2 = line.endpoints()
line.addCircle(c1)
line.addCircle(c3)
line.addCircle(c2)
pt1, pt2 = line.endpoints()
print(pt1, pt2)
m,b = line.val()
print(m,b)

In [ ]:
# Talk to the amcrest camera
from amcrest import AmcrestCamera
import os

if False:
    camera = AmcrestCamera('192.168.0.209', 80, 'honeybadger', 'DoucheBag').camera
    #print(camera.software_information)
    filename = "__amcrest_camera_.jpg"
    camera.snapshot(path_file=filename)
    imageCam = cv.imread(filename)
    os.remove(filename)
    ImgShow(imageCam)


In [ ]:
# Load the test image
imageOriginal = cv.imread("../test/images/long_all.jpg")

# Marker hints for the image
markers = [
    (1098,342), # top
    (1801,331),
    
    (558,660), #left
    (501,924),
    (430,1228),
    (270,1993),
    (176,2474),
    (54,3043),
    
    (2350,636), #right
    (2414,900),
    (2479,1198),
    (2654,1956),
    (2763,2437),
    (2895,2992),
    
    (799,3991), #bottom
    (1493,3981),
    (2187,3981)
]

# Pool tables are always twice as long as they are wide
# I don't think we will need this structure
poolTables = [
    #("7",  (44,88)),
    #("7+", (44,88)),
    ("8",   (44,88)),
    ("8+",  (46,92)),
    ("9",   (50,100)),
    ]


ImgShow(imageOriginal)


In [ ]:
# Blur the image before doing the mask
blur = int(21)
imageBlur = cv.GaussianBlur(imageOriginal, (blur,blur), cv.BORDER_DEFAULT)

ImgShow(imageBlur)

In [ ]:
# Build a mask using a circle around each marker
imageInput = imageOriginal
imageMarkerMask = np.zeros(imageInput.shape[:2], np.uint8)

circleMaskRadius = 60
clrWhite = (255,255,255)
for marker in markers:
    cv.circle(imageMarkerMask, marker, circleMaskRadius, clrWhite, -1)
    
# Apply the mask to the image
imageMarkers = cv.bitwise_and(imageInput, imageInput, mask=imageMarkerMask)
    
ImgShow([imageOriginal, imageMarkerMask, imageMarkers],250)

In [ ]:
# Hough Circles. 
imageInput = imageMarkers
print("imageInput.shape = {0}".format(imageInput.shape))

# Limit by size range and distance
cols = imageInput.shape[1]
rows = imageInput.shape[0]
imageGray = cv.cvtColor(imageInput, cv.COLOR_BGR2GRAY)
imageGray = cv.medianBlur(imageGray, 5)
hcircles = cv.HoughCircles(imageGray, cv.HOUGH_GRADIENT, minDist=100, dp=1,
                               param1=100, param2=10,
                               minRadius=5, maxRadius=35)

# Create an array of points
markerPoints = []
imageCircles = imageOriginal.copy()
if hcircles is not None:
    hcircles = np.uint16(np.around(hcircles))
    for i in hcircles[0, :]: 
        pt = [i[0], i[1]]
    
        # Filter out points ouside of image
        if pt[0] < 0 or pt[1] < 0:
            continue
        if pt[0] > cols-1 or pt[1] > rows-1:
            continue
    
        markerPoints.append(pt)
        cv.circle(imageCircles, pt, 12, Color(), 7)
        
print("markerPoints count = {0}".format(len(markerPoints)))
    
ImgShow([imageInput, imageCircles], 320)

In [ ]:
# Filter out circles that don't match the color we want
# We will assume that the circle centers should be whiteish

# Find the range
clrWhite = (240, 240, 240)
fuzz = 35

imageCirclesWhite = imageOriginal.copy()
imageCirclesWhiteBlack = np.zeros(imageOriginal.shape[:3],np.uint8)

clrMin, clrMax = ColorRange(clrWhite, fuzz)
clrMin = (clrMin[0]-1,clrMin[1]-1,clrMin[2]-1,)
clrMax = (clrMax[0]+1,clrMax[1]+1,clrMax[2]+1,)
print("color range = {0} - {1}".format(clrMin, clrMax))
circlesWhite = []
circRadius = 19
for pt in markerPoints:
    
    clr = imageOriginal[pt[1],pt[0]]
    if (clr > clrMin).all() and (clr < clrMax).all():
        circlesWhite.append(pt)
        clr = Color()
        cv.circle(imageCirclesWhite, pt, circRadius, clr, 6)
        cv.circle(imageCirclesWhiteBlack, pt, circRadius, clr, 6)
        #print("pt={0} clr={1}".format(pt, clr))
print("Prunage: {0} - {1}".format(len(markerPoints), len(circlesWhite)))

ImgShow([imageCirclesWhite, imageCirclesWhiteBlack], 220)

In [ ]:
# Group the markers into lines

# Find the top and bottom points by scanning all points
ymin = 10E10
ymax = 0
for pt in markerPoints:
    if pt[1] < ymin:
        ymin = pt[1]
    if pt[1] > ymax:
        ymax = pt[1]

# Sort the markers
markersL = []
markersR = []
markersT = []
markersB = []

yFuzz = int(imageOriginal.shape[0] / 20)    # search range
xmid = int(imageOriginal.shape[1] / 2)
for pt in markerPoints:
    if pt[1] < ymin + yFuzz:
        markersT.append(pt)
    else:
        if pt[1] > ymax - yFuzz:
            markersB.append(pt)
        else:
            if pt[0] < xmid:
                markersL.append(pt)
            else:
                markersR.append(pt)

# Sort by position, helpful for imputing new phantom markers later on
markersL.sort(key = lambda circ: circ[1])
markersR.sort(key = lambda circ: circ[1])
markersT.sort(key = lambda circ: circ[0])
markersB.sort(key = lambda circ: circ[0])

# Another view into it
markersTLBR = [markersT, markersL, markersB, markersR]

print("markersL", markersL)
print("markersR", markersR)
print("markersT", markersT)
print("markersB", markersB)


In [ ]:
# Show on an image just for sanity
imageMarkerLines = imageOriginal.copy()

def DrawMarkers(img, mks, clrMark=(0,255,0), clrLine = (0,0,255)):
    pt1 = mks[0]
    pt2 = mks[len(mks) - 1]
    
    # int
    pt1 = (int(pt1[0]), int(pt1[1]))
    pt2 = (int(pt2[0]), int(pt2[1]))
    
    cv.line(img, pt1, pt2, clrLine, 4)
    for pt in mks:
        pt = (int(pt[0]), int(pt[1]))
        #print(pt)
        cv.circle(img, pt, 25, clrMark, 4)
        
DrawMarkers(imageMarkerLines, markersL)
DrawMarkers(imageMarkerLines, markersR)
DrawMarkers(imageMarkerLines, markersT)
DrawMarkers(imageMarkerLines, markersB)
            
ImgShow([imageMarkerLines], 210)

In [ ]:
# Find the corner points of intersection

def FindLine(mks):
    pt1 = mks[0]
    pt2 = mks[len(mks) - 1]
    x1 = pt1[0]
    y1 = pt1[1]
    x2 = pt2[0]
    y2 = pt2[1]
    rise = float(float(y2) - float(y1))
    run  = float(float(x2) - float(x1))
    if run != 0.0:
        m = rise/run
    else:
        m = float(10E10)
    b = float(y1 - m * x1)
    return (m,b)


def FindIntersection(mksA, mksB):
    lineA = FindLine(mksA)
    lineB = FindLine(mksB)
    a = lineA[0]
    c = lineA[1]
    b = lineB[0]
    d = lineB[1]
    
    x = (d-c)/(a-b)
    y = a*x+c
    return (round(x),round(y))


ptTL = FindIntersection(markersL, markersT)
ptTR = FindIntersection(markersT, markersR)
ptBL = FindIntersection(markersL, markersB)
ptBR = FindIntersection(markersR, markersB)

imageCorners = imageMarkerLines.copy()
print(ptTL)
dia = 175
clrCorner = (0,0,255)
fill = 10
cv.circle(imageCorners, ptTL, dia, clrCorner, fill)
cv.circle(imageCorners, ptTR, dia, clrCorner, fill)
print(ptBL)
cv.circle(imageCorners, ptBL, dia, clrCorner, fill)
cv.circle(imageCorners, ptBR, dia, clrCorner, fill)

ImgShow([imageCorners], 200)

In [ ]:
# Perspective Transform
rows = imageOriginal.shape[0]
cols = imageOriginal.shape[1]


# Convert a scalar point to an array
def ToPtArray(ptIn):
    ptsOut = []
    x,y = ptIn
    ptsOut = [x,y]
    return ptsOut

# The transformed space should be in dimensions that have the 
# correct aspect ratio for a pool table (2:1)
if rows > cols:
    tgtDims = (cols, cols*2)
else:
    tgtDims = (rows*2, rows)
print("Target Dims = {0}".format(tgtDims))

# Build the array of src and dst points
ptsTransformSrc = np.array( [
    ToPtArray(ptTL),
    ToPtArray(ptTR), 
    ToPtArray(ptBR),
    ToPtArray(ptBL) ]
         ).astype(np.float32)

ptsTransformDst = np.array( [    
    [0,0],
    [tgtDims[0]-1, 0],
    [tgtDims[0]-1, tgtDims[1]-1],
    [0, tgtDims[1]-1]] 
         ).astype(np.float32)

# Calculate the transform matrix
matTxToFlat = cv.getPerspectiveTransform(ptsTransformSrc, ptsTransformDst)
matTxFromFlat = cv.getPerspectiveTransform(ptsTransformDst, ptsTransformSrc)


# Apply the warp transform
imageWarp = cv.warpPerspective(imageCorners, matTxToFlat, tgtDims)

print("Perspective Transform")
ImgShow([imageCorners, imageWarp],240)

In [ ]:
# Transform functions

def NPA(inp):
    # Convert to float array
    rows = []
    for row in inp:
        rowArray = []
        for c in row:
            rowArray.append(float(c))
        rows.append(rowArray)
        
    # Build a numpy array
    return np.array(rows).astype(np.float32)

def TxPoint(matTx, pt):
    c3 = matTx[2,0]*pt[0] + matTx[2,1]*pt[1] + matTx[2,2]
    m = [(matTx[0,0]*pt[0] + matTx[0,1]*pt[1] + matTx[0,2])/c3,
         (matTx[1,0]*pt[0] + matTx[1,1]*pt[1] + matTx[1,2])/c3]
    return m

def TxPoints(matTx, pts):
    out = []
    for pt in pts:
        a = TxPoint(matTx, pt)
        out.append(a)
    return out

matTx = NPA([[2,0.5,-100],
            [0,2,0],
            [0,0.005,1]])

ptTx = TxPoint(matTx, [100.0, 100.0])
print("ptTx",ptTx)
X = TxPoints(matTx, 
                [[100.0, 100.0],
                 [100.0, 100.0],
                 [100.0, 100.0],
                 [100.0, 100.0]])
print("X",X)
# Should be [100.00000074505806, 133.33333432674408]]

# We can go both ways, test to be sure
pt =  [500,1000]
print("pt", [500,1000])
ptTx = TxPoint(matTxToFlat, pt)
ptOrig = TxPoint(matTxFromFlat, ptTx)
print("ptFlat",ptTx)
print("ptOrig",ptOrig)

In [ ]:
# Add the missing middle markers
# Now that we can work in "flat space", it's easier to impute
# the 'missing' markers over the side pockets and end plaque
 
def FindMissingMiddle(mks):
    cnt = len(mks)
    if 0 != (cnt % 2):
        return mks # Only add if even count
    
    # Transform to flat space
    mks = TxPoints(matTxToFlat, mks)
    
    # Find the middle points
    idxL = int(len(mks)/2)-1
    idxR = idxL+1
    
    # Interpolate a point between the two
    ptA = mks[idxL]
    ptB = mks[idxR]
    dx = float(ptB[0]) - float(ptA[0])
    dy = float(ptB[1]) - float(ptA[1])
    ptMiddle = (int(ptA[0] + dx/2), int(ptA[1] + dy/2))
    mks.insert(idxR, ptMiddle)
    
    # Transform back to native space
    mks = TxPoints(matTxFromFlat, mks)
    return mks

markersL2 = FindMissingMiddle(markersL)
markersR2 = FindMissingMiddle(markersR)
markersT2 = FindMissingMiddle(markersT)
markersB2 = FindMissingMiddle(markersB)

imageMarkerMissing = imageOriginal.copy()
DrawMarkers(imageMarkerMissing, markersL2)
DrawMarkers(imageMarkerMissing, markersR2)
DrawMarkers(imageMarkerMissing, markersT2)
DrawMarkers(imageMarkerMissing, markersB)
DrawMarkers(imageMarkerMissing, markersB2)

ImgShow(imageMarkerMissing, 300)

In [ ]:
# Find and add the missing 'phantom' markers

def Distance(pt1, pt2):
    dx = float(pt2[0])-float(pt1[0])
    dy = float(pt2[1])-float(pt1[1])
    return math.sqrt(dx*dx + dy*dy)

def AverageDistance(mks):
    sum = 0.0
    xsum = 0.0
    ysum = 0.0
    for i in range(1, len(mks)):
        d = Distance(mks[i], mks[i-1])
        sum += d
        xsum += float(mks[i][0]) - float(mks[i-1][0])
        ysum += float(mks[i][1]) - float(mks[i-1][1])
    ave = sum / (float(len(mks)-1))
    dx = xsum / (float(len(mks)-1))
    dy = ysum / (float(len(mks)-1))
    return dx, dy

def AddPhantomPoints(mks):
    # Transform the marker positions into flat space
    mks = TxPoints(matTxToFlat, mks)
    dx, dy = AverageDistance(mks)
    last = len(mks) - 1
    pt0 = [mks[0][0] - dx, mks[0][1] - dy]
    ptN = [mks[last][0] + dx, mks[last][1] + dy]
    mks.insert(0, pt0)
    mks.append(ptN)
    
    # Back to native space
    mks = TxPoints(matTxFromFlat, mks)
    return mks
    
mksL = AddPhantomPoints(markersL2)
mksR = AddPhantomPoints(markersR2)
mksT = AddPhantomPoints(markersT2)
mksB = AddPhantomPoints(markersB2)


# Add the start/end 'phantom' markers

# The new 'phantom' markers are at the 


imageMarkerMissing = imageOriginal.copy()
DrawMarkers(imageMarkerMissing, mksL)
DrawMarkers(imageMarkerMissing, mksR)
DrawMarkers(imageMarkerMissing, mksT)
DrawMarkers(imageMarkerMissing, mksB)

ImgShow(imageMarkerMissing, 300)


In [ ]:
# Use the phantom markers for new side lines
# This is the 'playing area'

mksPaT = [mksL[0], mksR[0]]
mksPaB = [mksL[len(mksL)-1], mksR[len(mksR)-1]]
mksPaL = [mksT[0], mksB[0]]
mksPaR = [mksT[len(mksT)-1], mksB[len(mksB)-1]]

imageSides = imageOriginal.copy()

# Draw the original marker lines
DrawMarkers(imageSides, mksL)
DrawMarkers(imageSides, mksR)
DrawMarkers(imageSides, mksT)
DrawMarkers(imageSides, mksB)

# Draw the new play area lines in a different color
clrLine = (255,255,0)
clrMark= (255,0,0)
DrawMarkers(imageSides, mksPaT, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageSides, mksPaB, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageSides, mksPaL, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageSides, mksPaR, clrLine = clrLine, clrMark = clrMark)

ImgShow(imageSides, 300)


In [ ]:
# Find the play area corners

print(mksPaT)
ptTL = FindIntersection(mksPaL, mksPaT)
ptTR = FindIntersection(mksPaT, mksPaR)
ptBL = FindIntersection(mksPaL, mksPaB)
ptBR = FindIntersection(mksPaR, mksPaB)

imageCorners = imageOriginal.copy()

# Draw the new play area lines in a different color
clrLine = (255,255,0)
clrMark= (255,0,0)
DrawMarkers(imageCorners, mksPaT, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageCorners, mksPaB, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageCorners, mksPaL, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageCorners, mksPaR, clrLine = clrLine, clrMark = clrMark)

print(ptTL)
dia = 175
clrCorner = (0,255,0)
fill = 10
cv.circle(imageCorners, ptTL, dia, clrCorner, fill)
cv.circle(imageCorners, ptTR, dia, clrCorner, fill)
cv.circle(imageCorners, ptBL, dia, clrCorner, fill)
cv.circle(imageCorners, ptBR, dia, clrCorner, fill)

ImgShow([imageCorners], 220)

In [ ]:
# Create a new transform with our new four corner points

# Build the array of src and dst points
ptsTransformSrc = np.array( [
    ToPtArray(ptTL),
    ToPtArray(ptTR), 
    ToPtArray(ptBR),
    ToPtArray(ptBL) ]
         ).astype(np.float32)

ptsTransformDst = np.array( [    
    [0,0],
    [tgtDims[0]-1, 0],
    [tgtDims[0]-1, tgtDims[1]-1],
    [0, tgtDims[1]-1]] 
         ).astype(np.float32)

# Calculate the transform matrix
matTxToFlat = cv.getPerspectiveTransform(ptsTransformSrc, ptsTransformDst)
matTxFromFlat = cv.getPerspectiveTransform(ptsTransformDst, ptsTransformSrc)

# Apply the warp transform
imageWarp = cv.warpPerspective(imageCorners, matTxToFlat, tgtDims)
print("imageWarp", imageWarp.shape)

print("Perspective Transform")
ImgShow([imageCorners, imageWarp],240)




In [ ]:
# Build a mask for the playable area

polyPlayable = np.array([ptTL, ptTR, ptBR, ptBL])
print("polyPlayable", polyPlayable)
clr = (255,255,255)
imageBlack = np.zeros(imageOriginal.shape[:2], np.uint8)
imageMask = cv.fillPoly(imageBlack, [polyPlayable], color=clr)
imageFelt = cv.bitwise_and(imageOriginal, imageOriginal, mask=imageMask)

ImgShow([imageOriginal, imageFelt],250)
